In [1]:
import pandas as pd
import numpy as np 
import pylab as P 
# Import the module for plotting
import matplotlib.pyplot as plt
from pandas import Series,DataFrame,merge,concat

In [5]:
# data import
intradayData = pd.read_csv('~/psychSignalData/data/stocktwits_twitter_withretweets_intraday.csv')

In [2]:
# symbols
symbStocks = pd.read_csv('~/psychSignalData/data/symbols/stocks.csv')
# section symbols
secMat = pd.read_csv('~/psychSignalData/data/symbols/basic_materials.csv')
secCong = pd.read_csv('~/psychSignalData/data/symbols/conglomerates.csv')
secConsu = pd.read_csv('~/psychSignalData/data/symbols/consumer_goods.csv')
secFinan = pd.read_csv('~/psychSignalData/data/symbols/financial.csv')
secHealth = pd.read_csv('~/psychSignalData/data/symbols/healthcare.csv')
secIndus = pd.read_csv('~/psychSignalData/data/symbols/industrial_goods.csv')
secServ = pd.read_csv('~/psychSignalData/data/symbols/services.csv')
secTech = pd.read_csv('~/psychSignalData/data/symbols/technology.csv')
secUtil = pd.read_csv('~/psychSignalData/data/symbols/utilities.csv')

In [4]:
symbStocks.dtypes
symbStocks = symbStocks.ix[:,0:3]

In [3]:
intradayData.dtypes

NameError: name 'intradayData' is not defined

In [ ]:
# intraday data rows = 980,881,844
intradayData.info()

> Dataset merge on stock exchanges

In [22]:
intrStockSymb = merge(intradayData,symbStocks,left_on='SYMBOL',right_on = ' SYMBOL',how = "left")
intrStockSymb.head()

,SOURCE,SYMBOL,TIMESTAMP_UTC,BULLISH_INTENSITY,BEARISH_INTENSITY,BULL_MINUS_BEAR,BULL_SCORED_MESSAGES,BEAR_SCORED_MESSAGES,BULL_BEAR_MSG_RATIO,TOTAL_SCANNED_MESSAGES,EXCHANGE,SYMBOL,TITLE
0,stocktwits+twitter_withretweets,10GEN,2011-04-08T13:00:00Z,0,0,0,0,0,0,1,PRIVATE,10GEN,10gen
1,stocktwits+twitter_withretweets,10GEN,2011-04-08T13:05:00Z,0,0,0,0,0,0,1,PRIVATE,10GEN,10gen
2,stocktwits+twitter_withretweets,10GEN,2011-04-16T15:13:00Z,0,0,0,0,0,0,1,PRIVATE,10GEN,10gen
3,stocktwits+twitter_withretweets,10GEN,2011-04-16T15:15:00Z,0,0,0,0,0,0,1,PRIVATE,10GEN,10gen
4,stocktwits+twitter_withretweets,10GEN,2011-04-16T15:16:00Z,0,0,0,0,0,0,1,PRIVATE,10GEN,10gen


In [ ]:
intrStockSymb = intrStockSymb.ix[:,1:11]
intrStockSymb.head()

In [ ]:
# export the stock exchanges mereged dataset
intrStockSymb.to_csv('~/psychSignalData/output/DataExchange_intraday.csv', encoding='utf-8')

> Group Data by stock exchange

In [21]:
# Group By Stock Exchange
exchangeGrouped_in = intrStockSymb.groupby(intrStockSymb['EXCHANGE'])
sumByExchange_in = exchangeGrouped_in.sum()
print(sumByExchange_in.columns)
sumByExchange_in

NameError: name 'intrStockSymb' is not defined

In [ ]:
# statistic description for bull bear messages ratio by exchanges
# statByExchange_Ratio_in = exchangeGrouped_in['BULL_BEAR_MSG_RATIO'].describe()
# statByExchange_Ratio_in.head()

In [ ]:
# export the statistic description by exchanges
statByExchange_in = exchangeGrouped_in.describe()
# statByExchange_in.to_csv('/Users/jessia/psychSignalData/output/StatisticExchange_intraday.csv', encoding='utf-8')
statByExchange_in

> Group by Stock exchange and companies

In [ ]:
# Group By Stock Exchange and company
exComGrouped_in = intrStockSymb.groupby([intrStockSymb['EXCHANGE'],intrStockSymb['SYMBOL']])
sumByExCom_in = exComGrouped_in.sum()
print(sumByExCom_in.columns)
sumByExCom_in

In [ ]:
# output statistic results for exchanges and companies
stat_exCom_in = exComGrouped_in.describe()
# stat_exCom_in.to_csv('/Users/jessia/psychSignalData/output/StatisticExchangeCompany_intraday.csv', encoding='utf-8')

In [ ]:
# statistic description for bull bear messages ratio by exchanges
stat_exchange_Ratio_in = exchangeGrouped_in['BULL_BEAR_MSG_RATIO'].describe()
stat_exchange_Ratio_in.head()

> For the dataset grouped by stock exchange: do sentiment summary based on messages quantity

In [ ]:
# a table only contains the total quality of bullish, bearish and total messages for each exchange
messagesTable_in = sumByExchange_in.ix[:,[3,4,6]]
messagesTable_in.head()

In [ ]:
# neutral = total - bull - bear
neutralCounts_in = messagesTable_in['TOTAL_SCANNED_MESSAGES']-messagesTable_in['BULL_SCORED_MESSAGES']-messagesTable_in['BEAR_SCORED_MESSAGES']
# print (neutralCounts)

In [ ]:
# add new columns to the table. by using 'insert(idx, col_name, value)'
messagesTable_in.insert(2, 'NEUTRAL_SCORED_MESSAGES', neutralCounts_in)
messagesTable_in.head()

In [ ]:
# the percentage of bull messages 
bullPerc_in = messagesTable_in['BULL_SCORED_MESSAGES']/messagesTable_in['TOTAL_SCANNED_MESSAGES']
# the percentage of bear messages
bearPerc_in = messagesTable_in['BEAR_SCORED_MESSAGES']/messagesTable_in['TOTAL_SCANNED_MESSAGES']
neutralPerc_in = neutralCounts_in/messagesTable_in['TOTAL_SCANNED_MESSAGES']

In [ ]:
# change data format to %
bullPerc_in = bullPerc_in.apply(lambda x: format(x, '.2%'))
bearPerc_in = bearPerc_in.apply(lambda x: format(x, '.2%'))
neutralPerc_in = neutralPerc_in.apply(lambda x: format(x, '.2%'))

In [ ]:
# sentiment summary by stock exchange
sentiOutcome_in = messagesTable_in
sentiOutcome_in.insert(1,'BULL_PERC', bullPerc_in)
sentiOutcome_in.insert(3,'BEAR_PERC', bearPerc_in)
sentiOutcome_in.insert(5,'NEUTRAL_PERC', neutralPerc_in)
sentiOutcome_in.head()

In [ ]:
# export to a csv file
sentiOutcome_in.to_csv('~/psychSignalData/output/sentimentSummaryByExchange_intraday.csv', encoding='utf-8')

> Dataset merge on business sectors.

    1. material:   6,618,061;
    2. conglomerates:  291,255; 
    3. consumer_goods: 4,476,435; 
    4. financial:   15,843,759; 
    5. healthcare: 7,682,499; 
    6. industrial_goods: 2,703,620; 
    7. services: 10,100,739; 
    8. technology: 14,076,260; 
    9. utilities: 999,042;


In [6]:
# merge
# merge dataset right join to sector business
secMatMerge = merge(intradayData,secMat,on='SYMBOL',how = "right")
# secMatMerge.isnull()
secMatMerge = secMatMerge.dropna()
secMatMerge['BUSINESS_SECTOR'] = 'Material'
secMatMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6618061 entries, 0 to 6618060
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      float64
BEAR_SCORED_MESSAGES      float64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    float64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(7), object(6)
memory usage: 706.9+ MB


In [7]:
secCongMerge = merge(intradayData,secCong,on='SYMBOL',how = "right")
secCongMerge = secCongMerge.dropna()
secCongMerge['BUSINESS_SECTOR'] = 'Conglomerates'
secCongMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291255 entries, 0 to 291254
Data columns (total 13 columns):
SOURCE                    291255 non-null object
SYMBOL                    291255 non-null object
TIMESTAMP_UTC             291255 non-null object
BULLISH_INTENSITY         291255 non-null float64
BEARISH_INTENSITY         291255 non-null float64
BULL_MINUS_BEAR           291255 non-null float64
BULL_SCORED_MESSAGES      291255 non-null int64
BEAR_SCORED_MESSAGES      291255 non-null int64
BULL_BEAR_MSG_RATIO       291255 non-null float64
TOTAL_SCANNED_MESSAGES    291255 non-null int64
TITLE                     291255 non-null object
EXCHANGE                  291255 non-null object
BUSINESS_SECTOR           291255 non-null object
dtypes: float64(4), int64(3), object(6)
memory usage: 31.1+ MB


In [8]:
secConsuMerge = merge(intradayData,secConsu,on='SYMBOL',how = "right")
secConsuMerge = secConsuMerge.dropna()
secConsuMerge['BUSINESS_SECTOR'] = 'Consumer_goods'
secConsuMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4476435 entries, 0 to 4476434
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      float64
BEAR_SCORED_MESSAGES      float64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    float64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(7), object(6)
memory usage: 478.1+ MB


In [9]:
secFinanMerge = merge(intradayData,secFinan,on='SYMBOL',how = "right")
secFinanMerge = secFinanMerge.dropna()
secFinanMerge['BUSINESS_SECTOR'] = 'Financial'
secFinanMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15843759 entries, 0 to 15843758
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      float64
BEAR_SCORED_MESSAGES      float64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    float64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(7), object(6)
memory usage: 1.7+ GB


In [10]:
secHealthMerge = merge(intradayData,secHealth,on='SYMBOL',how = "right")
secHealthMerge = secHealthMerge.dropna()
secHealthMerge['BUSINESS_SECTOR'] = 'Healthcare'
secHealthMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7682499 entries, 0 to 7682498
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      float64
BEAR_SCORED_MESSAGES      float64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    float64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(7), object(6)
memory usage: 820.6+ MB


In [11]:
secIndusMerge = merge(intradayData,secIndus,on='SYMBOL',how = "right")
secIndusMerge = secIndusMerge.dropna()
secIndusMerge['BUSINESS_SECTOR'] ='Industrial_goods'
secIndusMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2703620 entries, 0 to 2703619
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      int64
BEAR_SCORED_MESSAGES      int64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    int64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(4), int64(3), object(6)
memory usage: 288.8+ MB


In [12]:
secServMerge = merge(intradayData,secServ,on='SYMBOL',how = "right")
secServMerge = secServMerge.dropna()
secServMerge['BUSINESS_SECTOR'] = 'Services'
secServMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10100739 entries, 0 to 10100738
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      float64
BEAR_SCORED_MESSAGES      float64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    float64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(7), object(6)
memory usage: 1.1+ GB


In [13]:
secTechMerge = merge(intradayData,secTech,on='SYMBOL',how = "right")
secTechMerge = secTechMerge.dropna()
secTechMerge['BUSINESS_SECTOR'] ='Technology'
secTechMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14076260 entries, 0 to 14076259
Data columns (total 13 columns):
SOURCE                    object
SYMBOL                    object
TIMESTAMP_UTC             object
BULLISH_INTENSITY         float64
BEARISH_INTENSITY         float64
BULL_MINUS_BEAR           float64
BULL_SCORED_MESSAGES      float64
BEAR_SCORED_MESSAGES      float64
BULL_BEAR_MSG_RATIO       float64
TOTAL_SCANNED_MESSAGES    float64
TITLE                     object
EXCHANGE                  object
BUSINESS_SECTOR           object
dtypes: float64(7), object(6)
memory usage: 1.5+ GB


In [14]:
secUtilMerge = merge(intradayData,secUtil,on='SYMBOL',how = "right")
secUtilMerge = secUtilMerge.dropna()
secUtilMerge['BUSINESS_SECTOR'] = 'Utilities'
secUtilMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999042 entries, 0 to 999041
Data columns (total 13 columns):
SOURCE                    999042 non-null object
SYMBOL                    999042 non-null object
TIMESTAMP_UTC             999042 non-null object
BULLISH_INTENSITY         999042 non-null float64
BEARISH_INTENSITY         999042 non-null float64
BULL_MINUS_BEAR           999042 non-null float64
BULL_SCORED_MESSAGES      999042 non-null int64
BEAR_SCORED_MESSAGES      999042 non-null int64
BULL_BEAR_MSG_RATIO       999042 non-null float64
TOTAL_SCANNED_MESSAGES    999042 non-null int64
TITLE                     999042 non-null object
EXCHANGE                  999042 non-null object
BUSINESS_SECTOR           999042 non-null object
dtypes: float64(4), int64(3), object(6)
memory usage: 106.7+ MB


In [15]:
# merge the sectors datasets together
sectorsData_intraday = concat([secMatMerge,secCongMerge,secConsuMerge,secFinanMerge,secHealthMerge,
                      secIndusMerge,secServMerge,secTechMerge,secUtilMerge],
                     keys=['Matrial','Conglomerates','Consumer_goods','Financial','Healthcare',
                           'Industrial_goods','Services','Technology','Utilities'],
                     names = ['SECTOR','Sector_Index'])
sectorsData_intraday.head()

SOURCE SYMBOL  \
SECTOR  Sector_Index                                           
Matrial 0             stocktwits+twitter_withretweets     AA   
        1             stocktwits+twitter_withretweets     AA   
        2             stocktwits+twitter_withretweets     AA   
        3             stocktwits+twitter_withretweets     AA   
        4             stocktwits+twitter_withretweets     AA   

                             TIMESTAMP_UTC  BULLISH_INTENSITY  \
SECTOR  Sector_Index                                            
Matrial 0             2009-08-24T17:15:00Z               0.00   
        1             2009-08-24T17:19:00Z               1.19   
        2             2009-09-03T13:50:00Z               0.00   
        3             2009-09-03T14:07:00Z               1.33   
        4             2009-09-10T14:27:00Z               2.50   

                      BEARISH_INTENSITY  BULL_MINUS_BEAR  \
SECTOR  Sector_Index                                       
Matrial 0                             0             0.00   
        1                             0             1.19   
        2                             0             0.00   
        3                             0             1.33   
        4                             0             2.50   

                      BULL_SCORED_MESSAGES  BEAR_SCORED_MESSAGES  \
SECTOR  Sector_Index                                               
Matrial 0                                0                     0   
        1                                1                     0   
        2                                0                     0   
        3                                1                     0   
        4                                1                     0   

                      BULL_BEAR_MSG_RATIO  TOTAL_SCANNED_MESSAGES       TITLE  \
SECTOR  Sector_Index                                                            
Matrial 0                               0                       1  Alcoa Inc.   
        1                               0                       1  Alcoa Inc.   
        2                               0                       1  Alcoa Inc.   
        3                               0                       1  Alcoa Inc.   
        4                               0                       1  Alcoa Inc.   

                     EXCHANGE BUSINESS_SECTOR  
SECTOR  Sector_Index                           
Matrial 0                NYSE        Material  
        1                NYSE        Material  
        2                NYSE        Material  
        3                NYSE        Material  
        4                NYSE        Material

In [ ]:
sectorsData_intraday.to_csv('~/psychSignalData/output/DataBySectors_intraday.csv', encoding='utf-8')

In [20]:
# count the available rows for each sector
sectorsData_intraday['SOURCE'].groupby(level = 'SECTOR').count()

SECTOR
Matrial              6618061
Conglomerates         291255
Consumer_goods       4476435
Financial           15843759
Healthcare           7682499
Industrial_goods     2703620
Services            10100739
Technology          14076260
Utilities             999042
dtype: int64